In [1]:
data = open("utterances_1.txt", "r").readlines()
len(data)

1870411

# Dataset Preparation

In [8]:
import pandas as pd
import numpy as np

In [4]:

ENTITIES = ["DEVICE","DEIVCE","Device","PREFERENCE","ACTION","VAL", "VALUE","UNIT","LOCATION","STATE","SERVICE:","SERVICE","TIME","ATION", "QUANTITY"]

sentence = []
labels = []
for line in data:
    split_lines = line.split('|||')    
    sentence.append(split_lines[0].strip())
    labels.append(split_lines[1].strip())

df = pd.DataFrame({'sentences':sentence,'labels':labels})

physical_parameters = [ "Color" ,"Temperature" ,"Brightness" ,"Volume" ,"AirQuality" ,"Humidity","Channel" ,"Noise" ,"EnergyConsumption","WaterConsumption"]
effect =    ["DecreaseValue" ,"FixedValue" ,"IncreaseValue"  ,"DEcreaseValue"]
goal =      ["Direct", "Indirect"]
scope =     ["Global","Device"]
df['EFFECT'] = df['labels'].str.findall('EFFECT:(\S*).*').str[0]
df['EFFECT'] = df['EFFECT'].str.replace('DEcreaseValue','DecreaseValue')

df['PARAM'] = df['labels'].str.findall('PARAM:(\S*).*').str[0]
df['SCOPE'] = df['labels'].str.findall('SCOPE:(\S*).*').str[0]
df['GOAL'] = df['labels'].str.findall('GOAL:(\S*).*').str[0]
df.head()


,sentences,labels,EFFECT,PARAM,SCOPE,GOAL
0,help me automatically ACTION:turn the LOCATION...,EFFECT:DecreaseValue PARAM:Colour SCOPE:Global...,DecreaseValue,Colour,Global,Indirect
1,please i need DEVICE:kettle LOCATION:out_here ...,SCOPE:Device GOAL:DeviceCommand,NaN,NaN,Device,DeviceCommand
2,automatically ACTION:lessen my DEVICE:smartpho...,EFFECT:FixedValue PARAM:Brightness SCOPE:Devic...,FixedValue,Brightness,Device,DeviceCommand
3,ACTION:suggest_us some VAL:loving for me,SCOPE:Global GOAL:Direct,NaN,NaN,Global,Direct
4,i prefer to have ACTION:make this DEVICE:windo...,SCOPE:Device GOAL:DeviceCommand,NaN,NaN,Device,DeviceCommand


In [5]:
u = df.sentences.str.split(expand=True).stack()

df_tokenized = pd.DataFrame({
    'Sentence': u.index.get_level_values(0) + 1, 
    'Word': u.values})

In [6]:
df_tokenized[['Word','Entity']] = df_tokenized['Word'].str.split(':',1,expand=True)
df_tokenized['Word_temp'] = df_tokenized['Word']

In [9]:
df_tokenized['Word'] = np.where(df_tokenized['Word_temp'].str.isupper(),df_tokenized['Entity'],df_tokenized['Word_temp'])
df_tokenized['Entity'] = np.where(df_tokenized['Word_temp'].str.isupper(),df_tokenized['Word_temp'],df_tokenized['Entity'])
df_tokenized = df_tokenized.drop('Word_temp',axis=1)
df_tokenized['Entity'] = df_tokenized['Entity'].fillna('O')
df_tokenized.head()

,Sentence,Word,Entity
0,1,help,O
1,1,me,O
2,1,automatically,O
3,1,turn,ACTION
4,1,the,O


In [10]:
len(df_tokenized), df_tokenized['Word'].nunique(), df_tokenized['Entity'].nunique()

(18648505, 2296, 16)

In [11]:
df_tokenized['Entity'].unique()

array(['O', 'ACTION', 'LOCATION', 'VAL', 'DEVICE', 'UNIT', 'TIME',
       'STATE', 'I', 'SERVICE', 'DEIVCE', 'amazon_echo', 'ATION',
       'PREFERENCE', 'MAHDAAAAAAAAAA', 'MAHDAAAAAAAA'], dtype=object)

In [12]:
df_tokenized['Word'] = np.where(df_tokenized['Entity']=='I','I',df_tokenized['Word'])
df_tokenized['Entity'] = np.where(df_tokenized['Entity']=='I','O',df_tokenized['Entity'])

In [13]:
df_tokenized['Word'] = np.where(df_tokenized['Entity']=='amazon_echo','amazon_echo',df_tokenized['Word'])
df_tokenized['Entity'] = np.where(df_tokenized['Entity']=='amazon_echo','Device',df_tokenized['Entity'])

In [14]:
df_tokenized['Entity'].unique()

array(['O', 'ACTION', 'LOCATION', 'VAL', 'DEVICE', 'UNIT', 'TIME',
       'STATE', 'SERVICE', 'DEIVCE', 'Device', 'ATION', 'PREFERENCE',
       'MAHDAAAAAAAAAA', 'MAHDAAAAAAAA'], dtype=object)

In [15]:
df_tokenized['Entity'] = np.where(df_tokenized['Entity'].isin(['Device','DEIVCE']),'DEVICE',df_tokenized['Entity'])
df_tokenized['Entity'] = np.where(df_tokenized['Entity']=='ATION','ACTION',df_tokenized['Entity'])

In [16]:
df_tokenized['Entity'].unique()

array(['O', 'ACTION', 'LOCATION', 'VAL', 'DEVICE', 'UNIT', 'TIME',
       'STATE', 'SERVICE', 'PREFERENCE', 'MAHDAAAAAAAAAA', 'MAHDAAAAAAAA'],
      dtype=object)

In [17]:
len(df_tokenized), df_tokenized['Word'].nunique(), df_tokenized['Entity'].nunique()

(18648505, 2297, 12)

In [18]:
df_tokenized.head()

,Sentence,Word,Entity
0,1,help,O
1,1,me,O
2,1,automatically,O
3,1,turn,ACTION
4,1,the,O


In [19]:
df_tokenized.groupby('Entity').size().reset_index(name='counts').sort_values(by='counts')

,Entity,counts
3,MAHDAAAAAAAA,6
4,MAHDAAAAAAAAAA,7
6,PREFERENCE,3836
9,TIME,64953
7,SERVICE,68128
8,STATE,212398
10,UNIT,368416
1,DEVICE,916244
11,VAL,1311821
2,LOCATION,1425964


In [20]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [21]:
df_tokenized = df_tokenized[(df_tokenized['Word'].str.len() !=0) & (df_tokenized['Word'].notnull())]

In [22]:
df_tokenized['Word'].isnull().sum()

0

In [23]:
unique_words = df_tokenized['Word'].unique()
unique_words = [w for w in unique_words if w is not None]

pos_tags={}
for word in unique_words: 
    doc= nlp(word)
    pos_tag = doc[0].pos_
    pos_tags[word] = pos_tag

In [24]:
df_tokenized['POS'] = df_tokenized['Word'].apply(lambda x: pos_tags[x] )

In [25]:
df_tokenized.head()

,Sentence,Word,Entity,POS
0,1,help,O,VERB
1,1,me,O,PRON
2,1,automatically,O,ADV
3,1,turn,ACTION,VERB
4,1,the,O,PRON


In [26]:
df_tokenized.to_csv("./data/tokenized_data_v1.csv",index=False)